## [Asynchronous I/O, event loop, and concurrency tools](https://pymotw.com/3/asyncio/index.html)

## Asynchronous Concurrency Concepts
**other concurrency model**: rely on threading or process management of the language runtime or operating system to change context

**asyncio**: explicitly handle context changes using eventloop and coroutine

**eventloop** - a place to register code to be run in application.

**coroutines** - as a mechanism for yielding control back to the event loop, these are special functions that give up control to their caller without losing their state (implented with generator in version older than Python 3.5)

**Future** - a data sturucture representing the result of work that has not been completed yet.

**Task** - a subclass of Future that knows how to wrap and manage the execution for a coroutine, can be scheduled with event loop.

## Cooperative Multitasking with Coroutines
Coroutines 
- a language construct designed for concurrency.
- creates a coroutine object when they called.
- then caller can run it using coroutine's `send()` method.
- while it is paused, their stats is manintaied
- when awakened, coroutines resume where it left off the next tinme.

### Starting a Coroutine

In [1]:
# asyncio_coroutine.py
import asyncio


async def coroutine():
    print('in coroutine')


event_loop = asyncio.get_event_loop()
try:
    print('starting coroutine')
    coro = coroutine()
    event_loop.run_until_complete(coro)
finally:
    print('closing event loop')
    # event_loop.close() - skipped in jupyter

starting coroutine
in coroutine
closing event loop


### Returning Values from Coroutines

In [2]:
# asyncio_coroutine_return.py
import asyncio


async def coroutine():
    print('in coroutine')
    return 'result'


event_loop = asyncio.get_event_loop()
try:
    return_value = event_loop.run_until_complete(
        coroutine()
    )
    print('it returned: {!r}'.format(return_value))
finally:
    pass
    # event_loop.close() - skipped in jupyter

in coroutine
it returned: 'result'


### Chaning Coroutines
decompising a task into resuable parts.

In [3]:
# asyncio_coroutine_chain.py
import asyncio


async def outer():
    print('in outer')
    print('waiting for result1')
    result1 = await phase1()
    print('waiting for result2')
    result2 = await phase2(result1)
    return (result1, result2)

async def phase1():
    print('in phase1')
    return 'result1'

async def phase2(arg):
    print('in phase2')
    return 'result2 derived from {}'.format(arg)

event_loop = asyncio.get_event_loop()
try:
    return_value = event_loop.run_until_complete(outer())
    print('return value: {!r}'.format(return_value))
finally:
    pass
    # event_loop.close() - skipped in jupyter

in outer
waiting for result1
in phase1
waiting for result2
in phase2
return value: ('result1', 'result2 derived from result1')


### Generators Instead of Coroutines
Earlier versions of Python 3 can use generator functions with the `asyncio.coroutine()` decorator and `yield from`

In [4]:
# asyncio_generator.py
import asyncio


@asyncio.coroutine
def outer():
    print('in outer')
    print('waiting for result1')
    result1 = yield from phase1()
    print('waiting for result2')
    result2 = yield from phase2(result1)
    return (result1, result2)


@asyncio.coroutine
def phase1():
    print('in phase1')
    return 'result1'


@asyncio.coroutine
def phase2(arg):
    print('in phase2')
    return 'result2 derived from {}'.format(arg)


event_loop = asyncio.get_event_loop()
try:
    return_value = event_loop.run_until_complete(outer())
    print('return value: {!r}'.format(return_value))
finally:
    pass
    # event_loop.close()

in outer
waiting for result1
in phase1
waiting for result2
in phase2
return value: ('result1', 'result2 derived from result1')


## Scheduling Calls to Regular Functions
schedule calls to regular functions based on the timer value in the loop.

In [5]:
# asyncio_call_soon.py
import asyncio
import functools


def callback(arg, *, kwarg='default'):
    print('callback invoked with {} and {}'.format(arg, kwarg))
    
async def main(loop):
    print('registering callbaks')
    loop.call_soon(callback, 1)
    wrapped = functools.partial(callback, kwarg='not default')
    loop.call_soon(wrapped, 2)
    
    await asyncio.sleep(0.1)
    
event_loop = asyncio.get_event_loop()
try:
    print('entering event loop')
    event_loop.run_until_complete(main(event_loop))
finally:
    print('closing event loop')
    # event_loop.close()

entering event loop
registering callbaks
callback invoked with 1 and default
callback invoked with 2 and not default
closing event loop


### Scheduling a Callback with a Delay

In [6]:
# asyncio_call_later.py

import asyncio

def callback(n):
    print('callback {} invoked'.format(n))
    
async def main(loop):
    print('registering callbacks')
    loop.call_later(0.2, callback, 1)
    loop.call_later(0.1, callback, 2)
    loop.call_soon(callback, 3)
    
    await asyncio.sleep(0.4)
    
event_loop = asyncio.get_event_loop()
try:
    print('entering event loop')
    event_loop.run_until_complete(main(event_loop))
finally:
    print('closing event loop')
    # event_loop.close()

entering event loop
registering callbacks
callback 3 invoked
callback 2 invoked
callback 1 invoked
closing event loop


### Scheduling a Callback for a Specific Time
Use `time()` method of event loop to start the internal state of clock; event loop uses a monotonic clock, not a wall-clock time, to ensure that value of "now" never regresses.

In [7]:
# asyncio_call_at.py

import asyncio
import time


def callback(n, loop):
    print('callback {} invoked at {}'.format(n, loop.time()))
    
    
async def main(loop):
    now = loop.time()
    print('clock time: {}'.format(time.time()))
    print('loop  time: {}'.format(now))
    
    print('registering callbacks')
    loop.call_at(now + 0.2, callback, 1, loop)
    loop.call_at(now + 0.1, callback, 2, loop)
    loop.call_soon(callback, 3, loop)
    
    await asyncio.sleep(0)
    

event_loop = asyncio.get_event_loop()
try:
    print('entering event loop')
    event_loop.run_until_complete(main(event_loop))
finally:
    print('closing event loop')
    # event_loop.close()

entering event loop
clock time: 1487515531.9017363
loop  time: 74538.17479271
registering callbacks
callback 3 invoked at 74538.175461376
closing event loop
